In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import re


In [57]:
# Tính toán Jaccard similarity
def jaccard_similarity(query, document):
    query_words = set(re.split(r'\s+', query.lower().strip()))
    doc_words = set(re.split(r'\s+', document.lower().strip()))
    intersection = query_words.intersection(doc_words)
    union = query_words.union(doc_words)
    return len(intersection) / len(union)

In [58]:
df = pd.read_csv('questions_answers.csv')

# Tách câu hỏi và câu trả lời
questions = df['question']
answers = df['answer']
print(questions)

0      Nguyên tắc xét tuyển của học viện năm nay ntn?...
1                       Câu hỏi thường gặp về tuyển sinh
2      Chương trình đào tạo tại miền Nam có khác với ...
3      Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...
4      Em đăng ký NV1 là ngành An toàn thông tin, NV2...
                             ...                        
567    Điểm chuẩn, chỉ tiêu ngành công nghệ internet ...
568    Điểm chuẩn, chỉ tiêu ngành công nghệ internet ...
569    Điểm chuẩn, chỉ tiêu ngành công nghệ internet ...
570    Điểm chuẩn, chỉ tiêu ngành công nghệ internet ...
571                                           Bạn là ai?
Name: question, Length: 572, dtype: object


In [59]:
# Tạo bộ từ điển các từ viết tắt
abbreviation_dict = {
    "cntt": "công nghệ thông tin",
    "attt": "an toàn thông tin",
    "iot": "công nghệ internet vạn vật",
    "fintech": "công nghệ tài chính",
    "cndpt": "công nghệ đa phương tiện",
    "ttdpt": "truyền thông đa phương tiện",
    "qtkd": "quản trị kinh doanh",
    "tmdt": "thương mại điện tử",
    "khmt": "khoa học máy tính",
    "clc": "chất lượng cao",
}

# Hàm để thay thế các từ viết tắt trong câu bằng từ đầy đủ
def replace_abbreviations(text, abbreviation_dict):
    words = text.split()
    new_words = []
    for word in words:
        if word in abbreviation_dict:
            new_words.append(abbreviation_dict[word])
        else:
            new_words.append(word)
    return ' '.join(new_words)

# Ví dụ sử dụng
user_input = "Tôi đang học cntt và muốn tìm hiểu về ngành clc"
processed_input = replace_abbreviations(user_input, abbreviation_dict)

print("Trước khi xử lý:", user_input)
print("Sau khi xử lý:", processed_input)


Trước khi xử lý: Tôi đang học cntt và muốn tìm hiểu về ngành clc
Sau khi xử lý: Tôi đang học công nghệ thông tin và muốn tìm hiểu về ngành chất lượng cao


In [60]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

remove_punctuation("Chi tiết về     Xét tuyển tài năng?@")

'Chi tiết về     Xét tuyển tài năng'

In [61]:
def to_lowercase(text):
    return text.lower()
to_lowercase('Chi tiết về     Xét tuyển tài năng')


'chi tiết về     xét tuyển tài năng'

In [62]:
def replace_comma(text):
    return " ".join(text.split(","))
replace_comma('chi tiết về     xét tuyển tài năng năm 2021, 2022,2023')


'chi tiết về     xét tuyển tài năng năm 2021  2022 2023'

In [63]:
from pyvi import ViTokenizer
def tokenizerText(text):
    return ViTokenizer.tokenize(text)
tokenizerText('chi tiết về     xét tuyển tài năng năm 2021  2022 2023')

'chi_tiết về xét tuyển tài_năng năm 2021 2022 2023'

In [64]:
tokenizerText("Học máy rất thú vị.")

'Học máy rất thú_vị .'

In [65]:
# Khởi tạo tập hợp để lưu trữ các từ dừng
stopwords = set()

# Đọc từng hàng trong file stopwords.txt
with open('stopwords.txt', 'r', encoding='utf-8') as fp:
    for line in fp:
        word = line.strip() # Loại bỏ khoảng trắng đầu và cuối dòng
        if word:  # Kiểm tra xem dòng không rỗng
            stopwords.add(word)

# In danh sách các từ dừng
print(stopwords)
print(len(stopwords))


{'biết_đâu_chừng', 'biết_mấy', 'là_thế_nào', 'cho_được', 'bao_lâu', 'bị', 'cho_thấy', 'biết_chừng_nào', 'bởi_đâu', 'biết', 'bỗng_thấy', 'bây_nhiêu', 'là_vì', 'biết_chắc', 'cho_ăn', 'ngay_khi_đến', 'dù_sao', 'biết_trước', 'biết_bao', 'bởi_nhưng', 'bỗng_nhiên', 'bấy_nay', 'cho_đến_khi', 'bỗng_nhưng', 'bỗng_đâu', 'dù_rằng', 'biết_mình', 'làm_cho', 'ngay_thật', 'cho_rồi', 'cho_đang', 'bởi_thế', 'ngay_bây_giờ', 'là_ít', 'bỗng_không', 'cho_rằng', 'là', 'là_nhiều', 'là_cùng', 'bấy_lâu', 'bỗng', 'cho_hay', 'cho_về', 'biết_bao_nhiêu', 'bấy_lâu_nay', 'làm_bằng', 'bởi_chưng', 'cho_biết', 'biết_được', 'cho_tin', 'ngay_cả', 'bỗng_dưng', 'bởi_vì', 'cho_tới_khi', 'cho_đến', 'cho_tới', 'biết_thế', 'dù_gì', 'cho_nhau', 'bởi_thế_cho_nên', 'biết_việc', 'bởi_sao', 'ngay_khi', 'cho_nên', 'ngay_lúc', 'biết_đâu_đấy', 'dù_dì', 'làm', 'bởi', 'bỗng_chốc', 'cho_chắc', 'ngay_lúc_này', 'là_phải', 'bởi_tại', 'bấy_nhiêu', 'ngay_từ', 'dù_cho', 'ngay_tức_thì', 'bây_giờ', 'ngay_tức_khắc', 'biết_đâu', 'bao_nhiêu', 'bấy_

In [66]:
def remove_stopwords(line):
    words = []
    for word in line.split():
        if word not in stopwords:
            words.append(word)
    return ' '.join(words)
text = 'Điểm chuẩn ngành cntt theo phương thức thi thpt 2023 là bao nhiêu?'
print(tokenizerText(text))
print(remove_stopwords(tokenizerText(text)))

Điểm_chuẩn ngành cntt theo phương_thức thi thpt 2023 là bao_nhiêu ?
Điểm_chuẩn ngành cntt theo phương_thức thi thpt 2023 ?


In [67]:
def processing_text_for_db(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_comma(text)
    text = tokenizerText(text)
    text = remove_stopwords(text)
    return text

In [68]:
def processing_text_for_query(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_comma(text)
    text = replace_abbreviations(text, abbreviation_dict)
    text = tokenizerText(text)
    text = remove_stopwords(text)
    return text

Demo preprocessing question

In [69]:
q = "Điểm chuẩn ngành cntt clc năm 2023 theo phương thức thi thpt tại cơ sở miền bắc   là bao   nhiêu ?!$"
q1 = "2021,2022,2023"
print(processing_text_for_query(q))
print(processing_text_for_query(q1))



điểm_chuẩn ngành công_nghệ thông_tin chất_lượng cao năm 2023 theo phương_thức thi thpt tại cơ_sở miền bắc
202120222023


In [70]:
processed_questions = []
for question in questions:  # Sử dụng tqdm để theo dõi tiến trình nếu cần
    # Áp dụng các hàm tiền xử lý
    processed_text = processing_text_for_db(question)
   
    # Lưu kết quả vào list processed_questions
    processed_questions.append(processed_text)

# Thêm cột mới vào DataFrame hoặc ghi đè lên cột 'question' hiện tại
df['processed_question'] = processed_questions

print(len(df))
# In ra vài dòng đầu của DataFrame sau khi đã tiền xử lý
print(df.head())

572
                                            question  \
0  Nguyên tắc xét tuyển của học viện năm nay ntn?...   
1                   Câu hỏi thường gặp về tuyển sinh   
2  Chương trình đào tạo tại miền Nam có khác với ...   
3  Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...   
4  Em đăng ký NV1 là ngành An toàn thông tin, NV2...   

                                              answer  \
0  I. Xét tuyển dựa vào kết quả thi THPT 2024:\n-...   
1                            Bạn đang gặp vấn đề gì?   
2  Chương trình đào tạo của Học viện được áp dụng...   
3  Chỉ tiêu đào tạo của từng cơ sở đã được Học vi...   
4  Không được em nhé! Theo nguyên tắc xét tuyển t...   

                                  processed_question  
0  nguyên_tắc xét tuyển của học_viện năm nay ntn ...  
1                   câu hỏi thường gặp về tuyển_sinh  
2  chương_trình đào_tạo tại miền nam có khác với ...  
3  trường mình có 2 cơ_sở vậy sẽ phân_bố học_sinh...  
4  em đăng_ký nv1 ngành an_toàn thông_tin nv2 n

In [71]:
# import torch
# from transformers import AutoModel, AutoTokenizer

# phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# # Function to encode each question into vector representation
# def encode_question(question):
#     # Tokenize the question
#     tokens = tokenizer.tokenize(question)
#     # Encode tokens to input_ids
#     input_ids = tokenizer.convert_tokens_to_ids(tokens)
#     # Convert to PyTorch tensor
#     input_ids = torch.tensor([input_ids])
#     # Get features from PhoBERT
#     with torch.no_grad():
#         features = phobert(input_ids)
#     # Return the representation vector of the sentence
#     return features[0].squeeze().tolist()

# df['question_vector'] = questions.apply(encode_question)
# print(df.head())



In [72]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Function to encode each question into vector representation
def encode_question(question):
   # Tokenize và chuyển đổi câu văn bản thành tensor
    tokens = tokenizer(question, return_tensors='pt', padding=True, truncation=True)

    # Sử dụng model để mã hóa câu văn bản thành vector
    with torch.no_grad():
        output = phobert(**tokens)

    # Lấy vector biểu diễn từ outputs của model
    return output.last_hidden_state.mean(dim=1).numpy()  # Trung bình các vector token

df['question_vector'] = df['processed_question'].apply(encode_question)
print(df.head())



Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                            question  \
0  Nguyên tắc xét tuyển của học viện năm nay ntn?...   
1                   Câu hỏi thường gặp về tuyển sinh   
2  Chương trình đào tạo tại miền Nam có khác với ...   
3  Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...   
4  Em đăng ký NV1 là ngành An toàn thông tin, NV2...   

                                              answer  \
0  I. Xét tuyển dựa vào kết quả thi THPT 2024:\n-...   
1                            Bạn đang gặp vấn đề gì?   
2  Chương trình đào tạo của Học viện được áp dụng...   
3  Chỉ tiêu đào tạo của từng cơ sở đã được Học vi...   
4  Không được em nhé! Theo nguyên tắc xét tuyển t...   

                                  processed_question  \
0  nguyên_tắc xét tuyển của học_viện năm nay ntn ...   
1                   câu hỏi thường gặp về tuyển_sinh   
2  chương_trình đào_tạo tại miền nam có khác với ...   
3  trường mình có 2 cơ_sở vậy sẽ phân_bố học_sinh...   
4  em đăng_ký nv1 ngành an_toàn thông_tin nv2 

In [73]:
fifth_element = df['question_vector'].iloc[324]
print(df['question'].iloc[324])
print(df['processed_question'].iloc[324])
print(df['processed_question'].iloc[313])

print(fifth_element.shape)
print(fifth_element)


Điểm chuẩn ngành quản trị kinh doanh theo phương thức thi thpt năm 2023 tại cơ sở miền bắc là bao nhiêu?
điểm_chuẩn ngành quản_trị kinh_doanh theo phương_thức thi thpt năm 2023 tại cơ_sở miền bắc
điểm_chuẩn ngành công_nghệ kỹ_thuật điện điện_tử theo phương_thức thi thpt năm 2023 tại cơ_sở miền bắc
(1, 768)
[[ 2.90340215e-01  1.36474580e-01 -1.56192333e-01 -1.76405996e-01
  -1.07408777e-01 -9.27470438e-03  4.02296819e-02  1.04929879e-01
  -2.11213946e-01  5.33469729e-02 -1.14897422e-01  2.63191462e-01
   3.47780474e-02 -4.78619225e-02 -8.68032724e-02  5.96747547e-02
  -3.73598821e-02  1.74119920e-01  3.22679400e-01 -1.24577299e-01
  -1.89968631e-01  3.97558250e-02  7.06855878e-02  3.83722484e-01
  -9.09027383e-02 -4.85458001e-02 -8.57239217e-02  2.58249521e-01
  -1.51735529e-01 -1.97447821e-01 -1.53860405e-01 -2.50946701e-01
  -1.30912393e-01  1.43464550e-01  4.58258003e-01  3.87457162e-01
  -4.31904681e-02 -6.25605658e-02  1.13886289e-01  7.60852844e-02
   4.43650484e-02  2.18973830e-0

In [74]:
query = "Điểm chuẩn ngành an toàn thông tin theo phương thức thi thpt 2023 miền bắc?"
processed_query = processing_text_for_query(query)
print(processed_query)
query_vector = encode_question(processed_query)
print(query_vector.shape)
print(query_vector)


điểm_chuẩn ngành an_toàn thông_tin theo phương_thức thi thpt 2023 miền bắc
(1, 768)
[[ 2.47866198e-01  8.75657946e-02 -1.72575355e-01 -1.28289014e-01
  -1.36528999e-01  2.49251686e-02  2.10783035e-02  1.29400179e-01
  -1.74124524e-01  8.21061879e-02 -1.68095723e-01  1.89115480e-01
   4.06486169e-02 -1.03111692e-01 -5.82254268e-02  4.59612384e-02
  -1.04514919e-01  2.47891799e-01  2.09378958e-01 -1.13384359e-01
  -2.05431148e-01  1.34707972e-01  8.07819441e-02  4.12992805e-01
  -9.81463119e-02 -1.18047111e-01 -4.29528728e-02  2.61191756e-01
  -1.53154209e-01 -1.34863913e-01 -1.52581677e-01 -2.25703314e-01
  -1.42768115e-01  1.45331517e-01  4.31122392e-01  4.21756595e-01
  -6.69411942e-02 -5.49872033e-02  1.53473541e-01  3.05534843e-02
  -1.26891593e-02  3.01260233e-01  6.55282801e-03 -1.20831057e-01
   2.02900425e-01  1.56172484e-01  1.58965588e-02  8.28259960e-02
   4.37981971e-02  2.17373222e-01  2.27199525e-01  2.87639171e-01
   4.94743623e-02  3.82300884e-01  1.01569042e-01  1.54131

In [84]:
query = "Điểm chuẩn ngành cntt năm 2023 cntt năm 2023 cntt năm 2023 cntt năm 2023 miền nam, bắc, trung theo thpt tại miền bắc "
processed_query = processing_text_for_query(query)
print(processed_query)
query_vector = encode_question(processed_query)

cosine_similarities = [cosine_similarity(
        query_vector, qv).flatten() for qv in df['question_vector']]

best_match_index = np.argmax(cosine_similarities)
print(np.max(cosine_similarities))



điểm_chuẩn ngành công_nghệ thông_tin năm 2023 công_nghệ thông_tin năm 2023 công_nghệ thông_tin năm 2023 công_nghệ thông_tin năm 2023 miền nam bắc trung theo thpt tại miền bắc
0.8565277


In [77]:
from sklearn.metrics.pairwise import cosine_similarity


# Calculate cosine similarity
cosine_similarities = cosine_similarity(query_vector,fifth_element)

# Print cosine similarities
print("Cosine similarities:")
print(cosine_similarities)




Cosine similarities:
[[0.6962985]]


In [78]:

questions_vector = df['question_vector']
processed_questions = df['processed_question']  
questions = df['question']
def get_response(user_query):
    processed_query = processing_text_for_query(user_query)
    query_vector = encode_question(processed_query)
    # Tính toán độ tương đồng cosine giữa câu truy vấn của người dùng và các câu hỏi trong database
    cosine_similarities = [cosine_similarity(
        query_vector, qv).flatten() for qv in questions_vector]
    jaccard_similarities = [jaccard_similarity(
        processed_query, q) for q in  df['processed_question']]

    # Kết hợp kết quả từ hai độ tương đồng
    alpha = 0.6
    beta = 0.4
    # Chuyển đổi jaccard_similarities thành mảng một chiều
    jaccard_array = np.array(jaccard_similarities).reshape(len(df['processed_question']),1)

    # Kết hợp điểm số từ hai độ tương đồng
    combined_scores = alpha * np.array(cosine_similarities) + beta * jaccard_array

    best_match_index = np.argmax(combined_scores)
    if np.max(combined_scores) > 0.6:
        print(np.max(combined_scores))
        print(processed_query)
        print(processed_questions[best_match_index])
        print(questions[best_match_index])
        return answers[best_match_index]
    else:
        return "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn."
user_input = "điểm chuẩn ngành iot theo phương thức thi thpt 2022  tại miền bắc??"
response = get_response(user_input)
print(response)

# # Chạy chatbot
# user_input = input("You: ")
# if user_input.lower() == "exit":
#     break
# response = get_response(user_input)
# print(f"Bot: {response}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.9009230920246669
điểm_chuẩn ngành công_nghệ internet vạn_vật theo phương_thức thi thpt 2022 tại miền bắc
điểm_chuẩn chỉ_tiêu ngành công_nghệ internet vạn_vật thi thpt 2022 tại miền bắc
Điểm chuẩn, chỉ tiêu ngành công nghệ internet vạn vật thi thpt 2022 tại miền bắc?
Cơ sở miền bắc của Học Viện không đào tạo ngành IoT
